In [1]:
import pandas as pd
import numpy as np 

In [2]:
# Load the data into a DataFrame
transactions = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='Transactions')

# Define a new list of column names
new_columns = ['transaction_id', 'product_id', 'customer_id', 'transaction_date', 'online_order', 'order_status', 'brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost', 'product_first_sold_date']

# Assign the new column names to the DataFrame
transactions.columns = new_columns

# Drop the first row of the DataFrame
transactions = transactions.drop(0)

# Reset the index of the DataFrame
transactions = transactions.reset_index(drop=True)

In [3]:
# Load the data into a DataFrame, skipping the first row
new_customer_lists = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='NewCustomerList', skiprows=1)

# Check the unique values in the past_3_years_bike_related_purchases column
print(new_customer_lists['past_3_years_bike_related_purchases'].unique())
customer_demographic = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='CustomerDemographic')
customer_add = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='CustomerAddress')

C:\Users\tamer\AppData\Local\Temp\ipykernel_16668\1294007739.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  new_customer_lists = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='NewCustomerList', skiprows=1)


[86 69 10 64 34 39 23 74 50 72 94 48 60 38 32 88 61 83 65  2 11 44 26  5
 78 19 71 84 45 62 70 27 76 58 73 24 79 52 29 14 85 59 12 55 97 87 75 51
 47 31 22 54 82 98  0 36 15 53  9  8 49 93 25 91 18 30 56 21 95  1 37 57
  3 66 13 42 89 90 28 92  6  4 40 16 96 99 33 68 67  7 35 46 41 17 81 63
 20 80 43 77]


In [4]:
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
1,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
2,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
3,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
4,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           20000 non-null  object
 1   product_id               20000 non-null  object
 2   customer_id              20000 non-null  object
 3   transaction_date         20000 non-null  object
 4   online_order             19640 non-null  object
 5   order_status             20000 non-null  object
 6   brand                    19803 non-null  object
 7   product_line             19803 non-null  object
 8   product_class            19803 non-null  object
 9   product_size             19803 non-null  object
 10  list_price               20000 non-null  object
 11  standard_cost            19803 non-null  object
 12  product_first_sold_date  19803 non-null  object
dtypes: object(13)
memory usage: 2.0+ MB


In [6]:
# Convert data types
transactions['transaction_id'] = transactions['transaction_id'].astype(int)
transactions['product_id'] = transactions['product_id'].astype(int)
transactions['customer_id'] = transactions['customer_id'].astype(int)
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
transactions['list_price'] = pd.to_numeric(transactions['list_price'])
transactions['standard_cost'] = pd.to_numeric(transactions['standard_cost'])

# Handle missing values
# Option 1: Fill in missing values with a default value
transactions['online_order'] = transactions['online_order'].fillna(False)

# Option 2: Interpolate missing values based on surrounding data
transactions['brand'] = transactions['brand'].interpolate(method='pad')

# Option 3: Drop rows or columns that contain missing values
transactions = transactions.dropna(axis=0, how='any')

# Remove duplicates
transactions = transactions.drop_duplicates()

# Check for and handle outliers
# Calculate z-scores for the list_price column
z_scores = (transactions['list_price'] - transactions['list_price'].mean()) / transactions['list_price'].std()

# Remove rows with z-scores greater than 3 or less than -3 (i.e., remove outliers)
transactions = transactions[(z_scores > -3) & (z_scores < 3)]

# Reset the index of the DataFrame
transactions = transactions.reset_index(drop=True)

In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19803 entries, 0 to 19802
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  int32         
 1   product_id               19803 non-null  int32         
 2   customer_id              19803 non-null  int32         
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  bool          
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [8]:
new_customer_lists.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.8750,0.743750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.1125,0.945625,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.0100,1.010000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.0875,1.087500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.6500,0.650000,4,4,1.703125


In [9]:
# Load the data into a DataFrame
new_customer_lists = pd.read_excel(r"KPMG_VI_New_raw_data_update_final.xlsx", sheet_name='NewCustomerList')

# Drop the unnecessary columns
new_customer_lists = new_customer_lists.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'])

# Define a new list of column names
new_columns = ['first_name', 'last_name', 'gender', 'past_3_years_bike_related_purchases', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car', 'tenure', 'address', 'postcode', 'state', 'country', 'property_valuation', 'rank', 'value']

# Assign the new column names to the DataFrame
new_customer_lists.columns = new_columns

In [10]:
new_customer_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   first_name                           1001 non-null   object
 1   last_name                            972 non-null    object
 2   gender                               1001 non-null   object
 3   past_3_years_bike_related_purchases  1001 non-null   object
 4   DOB                                  984 non-null    object
 5   job_title                            895 non-null    object
 6   job_industry_category                836 non-null    object
 7   wealth_segment                       1001 non-null   object
 8   deceased_indicator                   1001 non-null   object
 9   owns_car                             1001 non-null   object
 10  tenure                               1001 non-null   object
 11  address                              1001 n

In [11]:
# Drop unnecessary columns
new_customer_lists.drop(['rank', 'value'], axis=1, inplace=True)

# Handle missing values
new_customer_lists.dropna(subset=['last_name', 'DOB', 'job_title', 'job_industry_category'], inplace=True)

# Convert data types
new_customer_lists['past_3_years_bike_related_purchases'] = pd.to_numeric(new_customer_lists['past_3_years_bike_related_purchases'], errors='coerce')
new_customer_lists['tenure'] = pd.to_numeric(new_customer_lists['tenure'], errors='coerce')
new_customer_lists['property_valuation'] = pd.to_numeric(new_customer_lists['property_valuation'], errors='coerce')

# Standardize the "gender" column
new_customer_lists['gender'] = new_customer_lists['gender'].str.lower()  # Convert to lowercase for consistency

# Clean and validate the "DOB" column
new_customer_lists['DOB'] = pd.to_datetime(new_customer_lists['DOB'], errors='coerce')  # Convert to datetime, invalid entries will be set as NaT

# Clean and standardize the "postcode" column
new_customer_lists['postcode'] = new_customer_lists['postcode'].str.replace('\D', '', regex=True)  # Remove non-numeric characters

# Clean other columns if needed
# For example, standardize the "wealth_segment" column
new_customer_lists['wealth_segment'] = new_customer_lists['wealth_segment'].str.title()  # Capitalize the first letter of each word

# Save the cleaned dataset to a new file
new_customer_lists.to_csv('cleaned_customer_lists.csv', index=False)

In [12]:
new_customer_lists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 1000
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           716 non-null    object        
 1   last_name                            716 non-null    object        
 2   gender                               716 non-null    object        
 3   past_3_years_bike_related_purchases  715 non-null    float64       
 4   DOB                                  715 non-null    datetime64[ns]
 5   job_title                            716 non-null    object        
 6   job_industry_category                716 non-null    object        
 7   wealth_segment                       716 non-null    object        
 8   deceased_indicator                   716 non-null    object        
 9   owns_car                             716 non-null    object        
 10  tenure       

In [13]:
customer_demographic.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7


In [14]:
# Define a new list of column names
new_columns = ['transaction_id', 'product_id', 'customer_id', 'transaction_date', 'online_order', 'order_status', 'brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost', 'product_first_sold_date']

# Assign the new column names to the DataFrame
customer_demographic.columns = new_columns

# Drop the first row of the DataFrame
customer_demographic = customer_demographic.drop(0)

In [15]:
customer_demographic.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [16]:
customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           4000 non-null   object
 1   product_id               4000 non-null   object
 2   customer_id              3875 non-null   object
 3   transaction_date         4000 non-null   object
 4   online_order             4000 non-null   object
 5   order_status             3913 non-null   object
 6   brand                    3494 non-null   object
 7   product_line             3344 non-null   object
 8   product_class            4000 non-null   object
 9   product_size             4000 non-null   object
 10  list_price               3698 non-null   object
 11  standard_cost            4000 non-null   object
 12  product_first_sold_date  3913 non-null   object
dtypes: object(13)
memory usage: 406.4+ KB


In [17]:
# Convert data types
customer_demographic['transaction_id'] = customer_demographic['transaction_id'].astype(int)
customer_demographic['list_price'] = pd.to_numeric(customer_demographic['list_price'], errors='coerce')


# Handle missing values
# Option 1: Fill in missing values with a default value
customer_demographic['order_status'] = customer_demographic['order_status'].fillna('Unknown')
customer_demographic['brand'] = customer_demographic['brand'].fillna('Unknown')
customer_demographic['product_line'] = customer_demographic['product_line'].fillna('Unknown')
customer_demographic['list_price'] = customer_demographic['list_price'].fillna(customer_demographic['list_price'].mean())
customer_demographic.dropna(axis=0, inplace=True)

In [18]:
customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3792 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           3792 non-null   int32  
 1   product_id               3792 non-null   object 
 2   customer_id              3792 non-null   object 
 3   transaction_date         3792 non-null   object 
 4   online_order             3792 non-null   object 
 5   order_status             3792 non-null   object 
 6   brand                    3792 non-null   object 
 7   product_line             3792 non-null   object 
 8   product_class            3792 non-null   object 
 9   product_size             3792 non-null   object 
 10  list_price               3792 non-null   float64
 11  standard_cost            3792 non-null   object 
 12  product_first_sold_date  3792 non-null   object 
dtypes: float64(1), int32(1), object(11)
memory usage: 399.9+ KB


In [19]:
customer_add.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
3,4,0 Holy Cross Court,4211,QLD,Australia,9
4,5,17979 Del Mar Point,2448,New South Wales,Australia,4


In [20]:
# Define a new list of column names
new_columns = ['customer_id', 'address', 'postcode', 'state', 'country', 'property_valuation']

# Assign the new column names to the DataFrame
customer_add.columns = new_columns

# Drop the first row of the DataFrame
customer_add = customer_add.drop(0)

In [21]:
customer_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 1 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   object
dtypes: object(6)
memory usage: 187.6+ KB


In [23]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('cleaned_KPMG_VI_New_raw_data_update_final.xlsx', engine='xlsxwriter')

# Write each DataFrame to a separate sheet in the xlsx file
transactions.to_excel(writer, sheet_name='Sheet1', index=False)
new_customer_lists.to_excel(writer, sheet_name='Sheet2', index=False)
customer_demographic.to_excel(writer, sheet_name='Sheet3', index=False)
customer_add.to_excel(writer, sheet_name='Sheet4', index=False)

# Close the Pandas Excel writer and save the xlsx file
writer.save();

C:\Users\tamer\AppData\Local\Temp\ipykernel_16668\1880536705.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save();
